In [1]:
import cv2
import numpy as np
import torch
from fastsam import FastSAM, FastSAMPrompt
import time
import matplotlib.pyplot as plt
import random
from PIL import Image
import clip


In [2]:

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
model = FastSAM('/home/alexlin/FastSAM-x.pt')

image_path = '/home/alexlin/FastSAM/examples/farm.jpg'
image = cv2.imread(image_path)
height, width, _ = image.shape
print(height, width)



3024 4032


In [3]:
import matplotlib.cm as cm

colormap = cm.get_cmap('cool')

# interpolate color from 0.0 to 1.0
def get_color(value):
    value = min(1.0, max(0.0, value))
    return np.array((colormap(value))[:3]) * 255

print(get_color(0))

[          0         255         255]


/tmp/ipykernel_4013/715969173.py:3: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colormap = cm.get_cmap('cool')


In [4]:
everything_results = model(image, device=DEVICE, retina_masks=True, imgsz=1024, conf=0.4, iou=0.9)
prompt_process = FastSAMPrompt(image, everything_results, device=DEVICE)
results = prompt_process.get_formatted_results()

/home/alexlin/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED

0: 768x1024 53 objects, 125.4ms
Speed: 5.7ms preprocess, 125.4ms inference, 244.9ms postprocess per image at shape (1, 3, 1024, 1024)


In [5]:
print(len(results))
print(results[0].keys())
print(results[50]['bbox'])

53
dict_keys(['id', 'segmentation', 'bbox', 'score', 'area'])
tensor([0.0000e+00, 4.0054e+02, 4.0320e+03, 9.9266e+02, 6.8961e-01, 0.0000e+00], device='cuda:0')


In [6]:


overlay_image = prompt_process.visualize_mask_results(results)
bbox_overlay_image = prompt_process.visualize_bbox_results(results)
# resize image down to 1/2
resized_overlay_image = cv2.resize(overlay_image, (width//2, height//2))
resized_image = cv2.resize(image, (width//2, height//2))
resized_bbox_overlay_image = cv2.resize(bbox_overlay_image, (width//2, height//2))

cv2.imshow("original", resized_image)
cv2.imshow("overlay", resized_overlay_image)
cv2.imshow("bbox_overlay", resized_bbox_overlay_image)
cv2.waitKey(0)


cv2.destroyAllWindows()